In [86]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

In [87]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core.node_parser import SentenceSplitter


Settings.llm = OpenAI(model="gpt-4o-mini")
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads https://huggingface.co/BAAI/bge-small-en-v1.5
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
# Settings.embed_model = OpenAIEmbedding()

In [37]:
from llama_index.core import StorageContext, load_index_from_storage

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/lyft/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/lyft/index_store.json.


In [72]:
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("cloud")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [77]:
 # build index
lyft_index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context,
embed_model=Settings.embed_model)

# persist index
lyft_index.storage_context.persist(persist_dir="./storage/lyft")
# uber_index.storage_context.persist(persist_dir="./storage/uber")

lyft_engine = lyft_index.as_query_engine(similarity_top_k=10)

In [78]:
from llama_index.core.tools import QueryEngineTool

query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=lyft_engine,
        name="cloud",
        description=(
            "KT cloud 사용법 설명"
        ),
    ),

]

In [79]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

agent = ReActAgent(
    tools=query_engine_tools,
    llm=OpenAI(model="gpt-4o-mini"),
    # system_prompt="..."
)

# context to hold this session/state

ctx = Context(agent)

In [85]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream, ToolCall

handler = agent.run("기술지원 사이트 사용법", ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCall):
        print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: Korean. I need to use a tool to help me answer the question.
Action: cloud
Action Input: {'input': 'KT Cloud Web-UI 접속 방법'}
Observation: KT Cloud Web-UI에 접속하기 위해서는 제공된 URL로 접속한 후, ID와 초기 비밀번호를 입력하여 로그인해야 합니다. 로그인 후에는 우측 상단의 사용자 정보 변경 메뉴에서 패스워드를 변경하는 것이 필수입니다.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: KT Cloud Web-UI에 접속하기 위해서는 다음 단계를 따르세요:

1. 제공된 URL로 접속합니다.
2. ID와 초기 비밀번호를 입력하여 로그인합니다.
3. 로그인 후, 우측 상단의 사용자 정보 변경 메뉴에서 패스워드를 변경하는 것이 필수입니다.

이 과정을 통해 KT Cloud Web-UI에 안전하게 접속할 수 있습니다.
Call cloud with {'input': 'KT Cloud Web-UI 접속 방법'}
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: KT Cloud Web-UI에 접속하려면 제공된 URL로 접속한 후, ID와 초기 비밀번호를 입력하여 로그인합니다. 로그인 후에는 우측 상단의 사용자 정보 변경 메뉴에서 패스워드를 변경하는 것이 필수입니다.

### Make testing vector DB

In [108]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents and build index
documents = SimpleDirectoryReader(
    "./cloud_data"
).load_data()
index = VectorStoreIndex.from_documents(documents)

In [113]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# chroma_client = chromadb.EphemeralClient()        ### 휘발성
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("cloud")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, 
    transformations=[SentenceSplitter(chunk_size=300, chunk_overlap=10)],
    # embed_model=OpenAIEmbedding()
)


In [114]:
retriever = index.as_retriever()
results = retriever.retrieve("KT 클라우드 사용법은?")

for node in results:
    print(node.node.get_content())

NIPA 기술지원 게시판 사용자 메뉴얼 
I. KT Cloud Web-UI 접속 방법 
A. 제공된 URL 로 접속, ID 및 초기 비밀번호 입력하여 로그인 
 
B. 우측 상단 사용자 정보 변경 메뉴에서 패스워드 변경(필수)
NIPA 기술지원 게시판 사용자 메뉴얼 
목차 
 
I. KT Cloud Web-UI 접속 방법 
II. 게시판 접속 방법 
III. 게시판 이용 방법 
1. 게시판 홈 
2. 공지사항 
3. 기술지원
